### 导入需要的库

In [1]:
from keras.layers import Bidirectional, Dense, Embedding, Input, Lambda, LSTM, RepeatVector, TimeDistributed, Layer, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.layers.advanced_activations import ELU
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras import backend as K
from keras.models import Model
from scipy import spatial
from random import shuffle
import tensorflow as tf
import pandas as pd
import numpy as np
import codecs
import os

Using TensorFlow backend.


## 加载目录和文档
首先，我们将设置主目录和一些有关文本特征的变量。我们将序列长度设置为5-20，将词汇表中的最大单词数设置为270000(原270054)，我们将使用300维embeddings。最后，从txt加载文本。文本文件来源于新闻、网页、小说等，包含大约287万个句子(130万条符合长度要求)。

In [2]:
BASE_DIR = './data/'
TRAIN_DATA_FILE = BASE_DIR + 'train.txt'# 287+万条句子
GLOVE_EMBEDDING = BASE_DIR + 'sgns.renmin.bigram-char'#单词->300维embedding
VALIDATION_SPLIT = 0.2
MIN_SEQUENCE_LENGTH = 5  #最小序列长度5
MAX_SEQUENCE_LENGTH = 20 #最大序列长度20
MAX_NB_WORDS = 270000
EMBEDDING_DIM = 300 #embedding维度300

texts = [] #通过列表来存储句子
with codecs.open(TRAIN_DATA_FILE, encoding='utf-8') as f:
    reader = f.readline()
    while reader: #取出句子,存入texts
        if (len(reader.split()) <= MAX_SEQUENCE_LENGTH) and (len(reader.split()) >= MIN_SEQUENCE_LENGTH):
            texts.append(reader)
        reader = f.readline()
f.close()

n_sents = len(texts)
print('Found %s texts in train.txt' % n_sents) #训练用句子个数

Found 1372687 texts in train.txt


### 文本预处理
使用Keras的tokenizer和text_to_sequences函数预处理文本

In [3]:
tokenizer = Tokenizer(MAX_NB_WORDS+1, oov_token='unk') #Tokenizer是一个用于向量化文本，或将文本转换为序列（即单词在字典中的下标构成的列表，从1算起）的类
tokenizer.fit_on_texts(texts)
print('Found %s unique tokens' % len(tokenizer.word_index))

## **关键步骤** 若不能正常工作，丢弃OOV_Token
tokenizer.word_index = {e:i for e,i in tokenizer.word_index.items() if i <= MAX_NB_WORDS} # <= 从1开始
#tokenizer.word_index[tokenizer.oov_token] = MAX_NB_WORDS + 1
word_index = tokenizer.word_index #word到index的字典
index2word = {v: k for k, v in word_index.items()} #index到word的字典

sequences = tokenizer.texts_to_sequences(texts)#序列的列表，列表中每个序列对应于一段输入文本
shuffle(sequences) #打乱句子
data_1 = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH) #序列全部填充到25维，尾补0
print('Shape of data tensor:', data_1.shape)
NB_WORDS = (min(tokenizer.num_words, len(word_index))+1) #+1 for zero padding 
print('NB_WORDS:', NB_WORDS)

data_val = data_1[1000000:1200000]
data_train = data_1[:1000000]


Found 270054 unique tokens
Shape of data tensor: (1372687, 20)
NB_WORDS: 270001


In [4]:
print(word_index['unk'])

1


In [6]:
print(index2word[270001])

KeyError: 270001

### Word embeddings
使用预训练的中文 word embeddings(355776个)。创建一个矩阵，在词汇表中为每个单词对应一个embedding，然后我们将这个矩阵作为权重传递给我们模型的embedding layer 

In [6]:
embeddings_index = {}
unk_embedding = np.zeros(300)

#取出word及其对应的embeddings，存入字典embeddings_index
with codecs.open(GLOVE_EMBEDDING, encoding='utf-8') as f:
    line = f.readline()
    line = f.readline()
    
    while line: #取出句子,存入texts
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        unk_embedding = unk_embedding + coefs
        embeddings_index[word] = coefs
        line = f.readline()
f.close()
unk_embedding = unk_embedding / len(embeddings_index)
print('Found %s word vectors.' % len(embeddings_index))
#print(unk_embedding)

glove_embedding_matrix = np.zeros((NB_WORDS, EMBEDDING_DIM)) #申请0数组，(413501,300)
for word, i in word_index.items():
    if i < NB_WORDS+1: #+1 for 'unk' oov token 
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            glove_embedding_matrix[i] = embedding_vector
        else:
            # 在embeddings索引中找不到的单词，将是unk的embeddings
            #print('i=',i)
            glove_embedding_matrix[i] = unk_embedding
print('Null word embeddings: %d' % np.sum(np.sum(glove_embedding_matrix, axis=1) == 0))

Found 355776 word vectors.
Null word embeddings: 1


### VAE 模型
模型基于seq2seq架构，包含双向LSTM编码器和LSTM解码器。
通过 RepeatVector（max_len）函数，将每个时间步的潜在表示作为输入提供给解码器decoder。为了避免标签的独热码表示，我们使用tf.contrib.seq2seq.sequence_loss函数，它只需要单词索引作为标签（与embedding矩阵的输入相同）并在内部计算最终的softmax（所以 模型以具有线性激活的dense层结束）。 
可选地，“sequence_loss”允许使用采样的softmax，这有助于处理大型词汇表（例如，具有50k字词汇），但在此没有使用。这里使用的解码器与文中实现的解码器不同; 不是将context vector作为解码器的初始状态和预测的单词作为输入，而是在每个时间步处输入潜在表示z作为输入。

In [7]:
batch_size = 100
max_len = MAX_SEQUENCE_LENGTH
emb_dim = EMBEDDING_DIM
latent_dim = 64
intermediate_dim = 256
epsilon_std = 1.0
kl_weight = 0.01
num_sampled=500
act = ELU()


x = Input(shape=(max_len,)) #输入是按批量的40维向量(句子)
x_embed = Embedding(NB_WORDS, emb_dim, weights=[glove_embedding_matrix], input_length=max_len, trainable=False)(x)
h = Bidirectional(LSTM(intermediate_dim, return_sequences=False, recurrent_dropout=0.2), merge_mode='concat')(x_embed)
#h = Bidirectional(LSTM(intermediate_dim, return_sequences=False), merge_mode='concat')(h)
#h = Dropout(0.2)(h)
#h = Dense(intermediate_dim, activation='linear')(h)
#h = act(h)
#h = Dropout(0.2)(h)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
# 分别实例化这些层，以便以后重用
repeated_context = RepeatVector(max_len)
decoder_h = LSTM(intermediate_dim, return_sequences=True, recurrent_dropout=0.2)
decoder_mean = Dense(NB_WORDS, activation='linear')#softmax is applied in the seq2seqloss by tf #TimeDistributed()
h_decoded = decoder_h(repeated_context(z))
x_decoded_mean = decoder_mean(h_decoded)


# placeholder loss
def zero_loss(y_true, y_pred):
    return K.zeros_like(y_pred)

'''
#Sampled softmax
logits = tf.constant(np.random.randn(batch_size, max_len, NB_WORDS), tf.float32)
targets = tf.constant(np.random.randint(NB_WORDS, size=(batch_size, max_len)), tf.int32)
proj_w = tf.constant(np.random.randn(NB_WORDS, NB_WORDS), tf.float32)
proj_b = tf.constant(np.zeros(NB_WORDS), tf.float32)

def _sampled_loss(labels, logits):
    labels = tf.cast(labels, tf.int64)
    labels = tf.reshape(labels, [-1, 1])
    logits = tf.cast(logits, tf.float32)
    return tf.cast(
                    tf.nn.sampled_softmax_loss(
                        proj_w,
                        proj_b,
                        labels,
                        logits,
                        num_sampled=num_sampled,
                        num_classes=NB_WORDS),
                    tf.float32)
softmax_loss_f = _sampled_loss
'''


# 用于计算VAE损失的自定义层
class CustomVariationalLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)
        self.target_weights = tf.constant(np.ones((batch_size, max_len)), tf.float32)

    def vae_loss(self, x, x_decoded_mean):
        #xent_loss = K.sum(metrics.categorical_crossentropy(x, x_decoded_mean), axis=-1)
        labels = tf.cast(x, tf.int32)
        xent_loss = K.sum(tf.contrib.seq2seq.sequence_loss(x_decoded_mean, labels, 
                                                     weights=self.target_weights,
                                                     average_across_timesteps=False,
                                                     average_across_batch=False), axis=-1)
                                                     #softmax_loss_function=softmax_loss_f), axis=-1)#,
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        xent_loss = K.mean(xent_loss)
        kl_loss = K.mean(kl_loss)
        return K.mean(xent_loss + kl_weight * kl_loss)
    
    #编写一个call方法，来实现自定义层
    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        print(x.shape, x_decoded_mean.shape)
        loss = self.vae_loss(x, x_decoded_mean)
        self.add_loss(loss, inputs=inputs)
        # we don't use this output, but it has to have the correct shape:
        return K.ones_like(x)
    
def kl_loss(x, x_decoded_mean):
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    kl_loss = kl_weight * kl_loss
    return kl_loss

loss_layer = CustomVariationalLayer()([x, x_decoded_mean])
vae = Model(x, [loss_layer])
opt = Adam(lr=0.01) 
vae.compile(optimizer='adam', loss=[zero_loss], metrics=[kl_loss])
vae.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
(?, 40) (?, 40, 413501)

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 40, 300)      124050300   input_1[0][0]                    
__________________________________________________________

### 模型训练
通过keras.fit()训练100epochs。对于验证数据，传递相同的数组两次，因为此模型的输入和标签相同。
如果不使用“tf.contrib.seq2seq.sequence_loss”（或其他类似的函数），
将必须作为标签传递单词的one-hot码高维度序列(batch_size，seq_len，vocab_size)消耗大量内存。

In [ ]:
def create_model_checkpoint(dir, model_name):
    filepath = dir + '/' + model_name + ".h5" 
    directory = os.path.dirname(filepath)
    try:
        os.stat(directory)
    except:
        os.mkdir(directory)
    checkpointer = ModelCheckpoint(filepath=filepath, verbose=1, save_best_only=True)
    return checkpointer

checkpointer = create_model_checkpoint('models', 'vae_seq2seq_test_very_high_std')



vae.fit(data_train, data_train,
     shuffle=True,
     epochs=100,
     batch_size=batch_size,
     validation_data=(data_val, data_val), callbacks=[checkpointer])

#print(K.eval(vae.optimizer.lr))
#K.set_value(vae.optimizer.lr, 0.01)

vae.save('models/vae_lstm.h5')
#vae.load_weights('models/vae_lstm.h5')

Instructions for updating:
Use tf.cast instead.
Train on 2700000 samples, validate on 70000 samples
Epoch 1/100
    100/2700000 [..............................] - ETA: 649:32:59 - loss: 517.2673 - kl_loss: 0.0052